<h1 style="color:gray;text-align:center">Marketing To Frequent Flyers - MADEUP AIRLINES</h1>
<hr>
<h2 style="color:gray">Before we begin, let us discuss a bit about the data fields that we received and how these might be of interest for Marketing Teams.</h2>
<hr>

<table>
  <tr>
    <th>Feature</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>ID</td>
    <td>We can drop this field from our analysis as it does not hold any useful information.</td>
  </tr>
  <tr>
    <td>Balance</td>
    <td>A high balance represents customers who do not exhaust balance quickly or are planned travellers. Low might indicate people who travel infrequently or both travel and exhaust frequently</td>
  </tr>
  <tr>
    <td>Qual_miles</td>
    <td>Brings lots of perks if customers reach target. However, a high value represents high loyalty (not necessarily a frequent flyer as frequent trips are likely to be short and would result in NOT high Qual_miles) and vice versa.</td>
  </tr>
  <tr>
    <td colspan=2><b>All the ones mentioned below are for past 12 months unless specified otherwise</b></td>
  </tr>
  <tr>
    <td>cc1_miles, cc2_miles, cc3_miles</td>
    <td>Card holders and freq. flyers who are either individuals or small business units.</td>
  </tr>
  <tr>
    <td>Bonus_miles</td>
    <td>High values indicate people who spend high volume (moneywise) on their cards and vice versa</td>
  </tr>
  <tr>
    <td>Bonus_trans</td>
    <td>This tells us the frequency of spending.</td>
  </tr>
  <tr>
    <td>Flight_miles_12mo</td>
    <td>This should be positively correlated with `Qual_miles` and may or may not with `Balance` since balance can be low because customer exhausted all points at the time of taking this data.</td>
  </tr>
  <tr>
    <td>Flight_trans_12</td>
    <td>This simply tells us whether the customer should be put in high usage or low usage bucket.</td>
  </tr>
  <tr>
    <td>Days_since_enroll</td>
    <td>Although an indicator of loyalty, doesn't tell much whether the user is active or dormant, overall, since we have 12 months data at best.</td>
  </tr>
</table>

In [1]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
df.cc1_miles.unique()

In [ ]:
df_features = df.iloc[:,1:]

In [ ]:
df_features.describe()

<hr>
<h1 style="color:gray;text-align: center">DATA PREPARATION</h1>
<hr>
<div style="text-align:justify">
<h3>We normalize because we want to bring the data to a common scale. Otherwise, the dissimilarity/similarity will be over-influenced by higher valued columns and under-influenced by smaller valued columns</h3>
<p>1) Because the bins were already created for ccx_miles columns, I figure the disparity between intervals sizes is delibrately introduced by the data creator as he/she understood the domain. So, the values 1,2,3,4,5 should be assumed equally spaced</p>
<p>2) So, ruling out the alternative to convert each value 1,2,3,4,5 with average of interval limits, keep those values and apply min-max standardization. Since, the variation between ccx_miles variables is already neutralized by creating bins, we do not overdo it by again taking z-scores</p>
<p>3) Rest of the columns, except id (dropped) and award? (already binary, 0 or 1), will be normalized by taking z-scores</p>
</div>

In [ ]:
norm_cols = list(df_features.columns[:2])+list(df_features.columns[5:10])
min_max_cols = list(df_features.columns[2:5])

In [ ]:
df_features.columns[5:9]

In [ ]:
df_features.loc[:,norm_cols] = (df_features[norm_cols] - df_features[norm_cols].mean())/df_features[norm_cols].std()

In [ ]:
(df_features[norm_cols] - df_features[norm_cols].mean()).head()

In [ ]:
min_max_cols

In [ ]:
df_features.loc[:,min_max_cols]=(df_features[min_max_cols]-\
                                 df_features[min_max_cols].min())/(df_features[min_max_cols].max()-\
                                                                   df_features[min_max_cols].min())

In [ ]:
df_features.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corr = df_features.corr
corr()

In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
sns.heatmap(corr(),cmap='RdYlGn_r', linewidths=0.5, annot=True)

### We will use the above relations to improve our comments on findings of cluster analysis, if applicable

In [ ]:
from scipy.cluster import hierarchy

In [ ]:
z = hierarchy.linkage(df_features,metric='euclidean',method='ward')
fig, ax = plt.subplots(figsize=(15,8))
clusters = hierarchy.dendrogram(z, truncate_mode='lastp',p=50,orientation='left',leaf_font_size=8,show_contracted=True)
plt.title('Using Euclidean Distance and Ward\'s method')
plt.ylabel('Cluster Size')
plt.xlabel('Distance')
plt.axvline(x=65,linestyle=':',c='yellow')
plt.axvline(x=50,linestyle=':',c='yellow')
plt.show()

<hr>
<div style="text-align:justify">
<p>Looking at the Dendrogram above, it looks like `4` is a fairly decent choice for the `number of clusters`. This can be arrived at if we cut the dendrogram at about `65` units of distance. However, this is purely based on visual cues and not defined objectively. One might argue that `6` also looks like a good choice if we consider distance `50` (slightly > 50), which would not be wrong. We will arrive at a better guess after looking at the elbow-plots while running K-means.</p>
</div>

In [ ]:
labels_4 = hierarchy.cut_tree(z, n_clusters=[4])

In [ ]:
labels_4.shape

In [ ]:
df_features['labels_4'] = labels_4.reshape(1,labels_4.shape[0])[0]

In [ ]:
df_features.head()

In [ ]:
def df_for_plotting_centroids(df, label_col_name, n_clusters):
    assert n_clusters > 0
    temp = df[df[label_col_name] == 0].iloc[:,:-1].mean()
    result = pd.DataFrame({'features':temp.index})
    result['cluster_1'] = temp.values
    for i in range(1,n_clusters):
        result['cluster_%s'%(i+1)] = df[df[label_col_name] == i].iloc[:,:-1].mean().values
    trans = result.transpose()
    trans.columns = trans.iloc[0,:].values
    trans = trans.iloc[1:,:]
    trans['elements'] = trans.index.values
    return trans

In [ ]:
from pandas.plotting import parallel_coordinates

plot_4_centroids = df_for_plotting_centroids(df_features,'labels_4',4)
fig, ax = plt.subplots(figsize=(15,6))
ax.set_title("Cluster Centroids - From Euclidean-Ward")
parallel_coordinates(plot_4_centroids,"elements",color=('red', 'yellow', 'blue','green'))
plt.xticks(rotation=45)
plt.show()

<hr>
<h1 style="color:gray;text-align: center">INTERPRETATION</h1>
<hr>
<div style="text-align:justify">
<p><span style="font-size:20px;color:red">Cluster 1 (Cheapest Price Seekers + Rare Travels): </span>This cluster has the least values in all features. These can represent customers who travel quite infrequently and decide on whatever airline offers lowest price.</p>
<p><span style="font-size:20px;color:#ffcc00">Cluster 2 (Budget Travellers): </span>With High `Balance`, Low `Qual_miles`, High `cc1_miles`, High `Bonus_miles`, but an overall low, they represent budget travellers. They seem to travel moderately frequently maybe for planned vacations, but when possible they seek out to accumulate and utilize points.</p>
<p><span style="font-size:20px;color:blue">Cluster 3 (Luxury Travellers): </span>With only high `Qual_miles`, these represent business travellers or retired people for whom comfort is the prime concern.</p>
<p><span style="font-size:20px;color:green">Cluster 4 (Reward Seekers): </span>With high `Bonus_miles`, `Bonus_trans`, `Flight_miles_12mo`, `Flight_trans_12`, they represent customers who are looking to collect as much points as they can, through travel or non-travel transactions. Also, they are long-haul frequent travellers, who sometimes also travel only to increase rewards and maybe become eligible for better deals.</p>
</div>

In [ ]:
df_features_95 = df_features.iloc[:,:-1].sample(frac=.95)
df_features_95.reset_index(drop=True,inplace=True)
df_features_95.head()

In [ ]:
df_features_95.shape

In [ ]:
z_95 = hierarchy.linkage(df_features_95,metric='euclidean',method='ward')
fig, ax = plt.subplots(figsize=(15,8))
clusters = hierarchy.dendrogram(z_95, truncate_mode='lastp',p=50,orientation='left',leaf_font_size=8,show_contracted=True)
plt.title('Using Euclidean Distance and Ward\'s method (95%)')
plt.ylabel('Cluster Size')
plt.xlabel('Distance')
plt.axvline(x=65,linestyle=':',c='yellow')
plt.axvline(x=50,linestyle=':',c='yellow')
plt.show()

In [ ]:
labels_4_95 = hierarchy.cut_tree(z_95, n_clusters=[4])
df_features_95['labels_4'] = labels_4_95.reshape(1,labels_4_95.shape[0])[0]

In [ ]:
plot_4_centroids_95 = df_for_plotting_centroids(df_features_95,'labels_4',4)

fig, ax = plt.subplots(figsize=(15,6))
ax.set_title("Cluster Centroids - From Euclidean-Ward (95%)")
parallel_coordinates(plot_4_centroids_95,"elements",color=('red', 'yellow', 'blue','green'))
plt.xticks(rotation=45)
plt.show()

<hr>
<div style="text-align:justify">
<p><span style="font-size:16px;">We notice very small change in Cluster 1 centroid values and that's all. The cluster count, as seen from dendrogram is `4` and `6`, if we cut at `65` and `50`, respectively, remains the same. The clustering seems pretty stable!<span></p>

</div>

In [ ]:
from sklearn.cluster import KMeans

df_for_kmeans = df_features.iloc[:,:-1]
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_for_kmeans)
    df_for_kmeans["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_

fig, ax = plt.subplots(figsize=(10,4))
plt.plot(list(sse.keys()), list(sse.values()))
plt.title("SCREE PLOT")
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

<hr>
<h1 style="color:gray;text-align: center">How many clusters?</h1>
<hr>
<div style="text-align:justify">
<p><span style="font-size:16px;">The plot looks sloping down quite gradually. This gradual nature makes deciding cluster count difficult, by visual inspection. When compared to other points, elbow at `2` and `4` seem most prominent. We can choose `4` to be the optimal cluster size and go ahead.<span></p>
</div>

In [ ]:
kmeans = KMeans(n_clusters=4, max_iter=1000).fit(df_for_kmeans)
df_for_kmeans["clusters"] = kmeans.labels_

In [ ]:
df_for_kmeans.head()

In [ ]:
kmeans_centroids = df_for_plotting_centroids(df_for_kmeans,'clusters',4)
kmeans_centroids

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
parallel_coordinates(kmeans_centroids,"elements",color=('red', 'yellow', 'blue','green'))
plt.xticks(rotation=45)
plt.show()

<hr>
<div style="text-align:justify">
    <p><span style="font-size:16px;">The clusters <b>do not seem to have changed</b> moving from Hierarchical to K-Means using `4` as optimal cluster size.</span></p>
</div>